<a href="https://colab.research.google.com/github/nicrandy/YahtzeeGamePlayAndAI/blob/main/YahtzAIGamePlay2_(version_working_on_as_of_Sep_24)_finished_up_to_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial dice, new dice, convert dice


In [1]:
import random
import csv
rollsRemaining = 2



#create a random array of five dice
def initial_array():
  diceArray = [0,0,0,0,0]
  i = 0
  while i < 5:
    diceArray[i] = random.randint(1, 6) #assign a random number from 1-6 for each of the five dice
    i = i + 1
  return diceArray

#assign new dice to rerolled dice
def get_new_dice(arrayIn):
  newArray = arrayIn
  i = 0
  while i < 5:
    if newArray[i] == 0:
      newArray[i] = random.randint(1, 6)
    i += 1
  return newArray

#take in the dice array and convert to the number of each dice array
def convert_diceArray_to_faceCountArray(diceScoresIn):
  ones = 0
  twos = 0
  threes = 0
  fours = 0
  fives = 0
  sixes = 0
  diceFacesCountArray = [0,0,0,0,0,0]
  i = 0
  while i < 5:
    if diceScoresIn[i] == 1:
      ones +=1
    if diceScoresIn[i] == 2:
      twos +=1
    if diceScoresIn[i] == 3:
      threes +=1
    if diceScoresIn[i] == 4:
      fours +=1
    if diceScoresIn[i] == 5:
      fives +=1
    if diceScoresIn[i] == 6:
      sixes +=1
    i += 1
  i = 0
  diceFacesCountArray = [ones,twos,threes,fours,fives,sixes]
  return diceFacesCountArray

def convert_faceCountArray_to_diceArray(faceCountArrayIn):
  diceArray = [0,0,0,0,0]
  dicePositionTracker = 0
  i = 0
  while i < 6:
    if faceCountArrayIn[i] >= 0:
      toAdd = faceCountArrayIn[i]
      j = 0
      while j < toAdd:
        diceArray[dicePositionTracker] = i + 1
        dicePositionTracker += 1
        j += 1
      j = 0
    i += 1
  return diceArray

#convert the dice face count array to either 1 = have that dice or 0 = dont have that dice
def convert_faceCountArray_to_haveDontHaveArray(diceFaceCountArray):
	i = 0
	diceFaceHaveArray = diceFaceCountArray
	while i < 6:
		if diceFaceHaveArray[i] > 1:
			diceFaceHaveArray[i] = 1
		i += 1
	return diceFaceHaveArray


# Probabilities


In [2]:
#for the upper section, 3 of a kind, 4 of a kind, and yahtzee
upperSectionAndKindsNextRoll = [[0.4020191, 0.401891, 0.1606186, 0.0321412, 0.0031973, 0.0001328], 
                                [0.0, 0.4823503, 0.3858599, 0.1155694, 0.0154578, 0.0007626], 
                                [0.0, 0.0, 0.5785395, 0.3472483, 0.0696254, 0.0045868], 
                                [0.0, 0.0, 0.0, 0.6945422, 0.2777333, 0.0277245], 
                                [0.0, 0.0, 0.0, 0.0, 0.83344, 0.16656],
                                [0,0,0,0,0,1]]
upperSectionAndKindsTwoRolls = [[0.1615261, 0.355305, 0.3124722, 0.1376844, 0.0303194, 0.0026929], 
                                [0.0, 0.2326, 0.4093246, 0.2700771, 0.0792911, 0.0087072], 
                                [0.0, 0.0, 0.3347088, 0.4421599, 0.1946394, 0.0284919], 
                                [0.0, 0.0, 0.0, 0.4822744, 0.4242362, 0.0934894], 
                                [0.0, 0.0, 0.0, 0.0, 0.694551, 0.305449],
                                [0,0,0,0,0,1]]
#for full house
#[0] is nothing, [1] is one pair, [2] is three of a kind, [3] is two pairs, [4] if full house
fullHouseNextRoll = [0.038734, 0.0924753, 0.1390113, 0.3333942, 1]
fullHouseTwoRolls = [0.1862441, 0.2598995, 0.2585258, 0.5555052, 1]

#for small straight
#[i] indicates the starting state, eg [4] is starting state 5
smallStraightNextRoll = [0, 0.10075879177002772, 0.14792492466207344, 0.1393458324096414, 
                         0.1508339934260263, 0.25047161620094677, 0.30676430021435214, 
                         0.32043807840730937, 0.5554347305580386, 1.0, 1.0, 0]
smallStraightTwoRolls = [0, 0.3414338377958867, 0.3914621131270011, 0.3542106075415402, 
                         0.3853223684417105, 0.5150658838511324, 0.5204131471022932, 
                         0.5468981432589293, 0.8020502673178741, 1.0, 1.0, 0]

largeStraightNextRoll = [0, 0.016941302652106083, 0.03533148074632791, 0.027452765258320196, 
                         0.05550147314736643, 0.055543672476412086, 0.11163400759762403, 
                         0.1666214775575335, 0.3342406890718723, 1.0, 0, 0]
largeStraightTwoRolls = [0, 0.09600524857850999, 0.1445275062296339, 0.11506027442919838, 
                         0.1853245085619242, 0.1635046235943654, 0.27833949674880376, 
                         0.30553063616187176, 0.5548873984121993, 1.0, 0, 0]

# Dice faces


In [3]:
#take in number of each face, return weighted probability of each face
def faces_weighted_scores(diceIn):
  faceCounts = convert_diceArray_to_faceCountArray(diceIn)
  weightedProbabilities = [0,0,0,0,0,0]
  i = 0 #track which dice face. eg. 1, 2, 3....
  while i < 6: #loop through the dice face counts 1 - 6
    numberOfThisDice = faceCounts[i]
    thisFaceWeight = 0
    diceFace = i + 1
    j = 0
    while j < 6: #loop through the weighted probabilities for that number of dice faces
      if rollsRemaining == 1: # for one reroll remaining
        thisFaceWeight += (upperSectionAndKindsNextRoll[numberOfThisDice][j] * j *(diceFace))
      if rollsRemaining == 2: # for two rerolls remaining
        thisFaceWeight += (upperSectionAndKindsTwoRolls[numberOfThisDice][j] * j *(diceFace))
      j += 1
    weightedProbabilities[i] = thisFaceWeight
    i += 1
  return weightedProbabilities

# return a 5x6 array of potential dice to keep of each face (array of ones to keep, array of twos to keep...)
def keep_desired_face(diceIn):
  numberOfEachFace = convert_diceArray_to_faceCountArray(diceIn)
  diceKeepArrays = [0,0,0,0,0,0]
  thisArray = [0,0,0,0,0]
  j = 0
  while j < 6:
    i = 0
    while i < numberOfEachFace[j]:
      thisArray[i] = j+1
      i+=1
    diceKeepArrays[j] = thisArray
    thisArray = [0,0,0,0,0]
    j+=1
  return diceKeepArrays
##### testing ###########################################
# dice = initial_array()
# print (dice)
# print (faces_weighted_scores(dice))
# rollsRemaining = 1
# print (faces_weighted_scores(dice))
# print (keep_desired_face(dice))




# 3x, 4x, Yahtzee

In [4]:
def kinds_highest_number_of_faces_and_face_total(diceIn):
  numberOfEachArray = convert_diceArray_to_faceCountArray(diceIn)
  maxFaceCount = 0
  maxFace = 0
  i = 0
  while i < 6:
    if numberOfEachArray[i] >= maxFaceCount:
      maxFaceCount = numberOfEachArray[i]
      maxFace = i+1
    i += 1
  return [maxFaceCount,maxFace]

#keep the most of the dice faces, return array with faces of the most, and set other dice to 0 to reroll
def kinds_keep_desired_dice(diceIn):
  threeKindKeeping = [0,0,0,0,0]
  maxFaceCount, maxFace = kinds_highest_number_of_faces_and_face_total(diceIn)
  i = 0
  while i < maxFaceCount:
    threeKindKeeping[i] = maxFace
    i += 1
  return threeKindKeeping

def three_kind_score_weight(diceIn):
  diceKeeping = [0,0,0,0,0]
  values = kinds_highest_number_of_faces_and_face_total(diceIn)
  maxFaceCount = values[0]
  maxFace = values[1]
  i = 0
  if maxFaceCount >= 3:
    i = 0
    while i < 5:
      if diceIn[i] == maxFace:
        diceKeeping[i] = diceIn[i]
      else:
        if rollsRemaining == 2:
          if diceIn[i] >= 5:
            diceKeeping[i] = diceIn[i]
          else:
            diceKeeping[i] = 4.25 #average score if reroll dice two times
        else: 
          if diceIn[i] >= 4:
            diceKeeping[i] = diceIn[i]
          else:
            diceKeeping[i] = 3.5 #average score if reroll dice one time
      i += 1
  if maxFaceCount < 3:
    i = 0
    while i < 3:
      diceKeeping[i] = maxFace #assign dice to face of dice keeping
      i += 1
    i = 3
    while i < 5:
      diceKeeping[i] = 3.5 #assign remaining dice to reroll average
      i += 1
  i = 0
  weightedScore = 0
  while i < 5:
    weightedScore += diceKeeping[i]
    i += 1
  if maxFaceCount >=3:
    return weightedScore
  else:
    scoreAdder = 0
    i = 3
    while i < 5:
      if rollsRemaining == 2:
        scoreAdder += upperSectionAndKindsTwoRolls[maxFaceCount][i] * weightedScore
      if rollsRemaining == 1:
        scoreAdder += upperSectionAndKindsNextRoll[maxFaceCount][i] * weightedScore
      i += 1
    return scoreAdder


def four_kind_score_weight(diceIn):
  diceKeeping = [0,0,0,0,0]
  values = kinds_highest_number_of_faces_and_face_total(diceIn)
  maxFaceCount = values[0]
  maxFace = values[1]
  if maxFaceCount >= 4:
    i = 0
    while i < 5:
      if diceIn[i] == maxFace:
        diceKeeping[i] = diceIn[i]
      else:
        if rollsRemaining == 2:
          if diceIn[i] >= 5:
            diceKeeping[i] = diceIn[i]
          else:
            diceKeeping[i] = 4.25 #average score if reroll dice two times
        else: 
          if diceIn[i] >= 4:
            diceKeeping[i] = diceIn[i]
          else:
            diceKeeping[i] = 3.5 #average score if reroll dice one time
      i += 1
  if maxFaceCount < 4:
    i = 0
    while i < 4:
      diceKeeping[i] = maxFace #assign dice to face of dice keeping
      i += 1
    i = 4
    while i < 5:
      diceKeeping[i] = 3.5 #assign remaining dice to reroll average
      i += 1
  i = 0
  weightedScore = 0
  while i < 5:
    weightedScore += diceKeeping[i]
    i += 1
  if maxFaceCount >=4:
    return weightedScore
  else:
    scoreAdder = 0
    i = 4
    while i < 5:
      if rollsRemaining == 2:
        scoreAdder += upperSectionAndKindsTwoRolls[maxFaceCount][i] * weightedScore
      if rollsRemaining == 1:
        scoreAdder += upperSectionAndKindsNextRoll[maxFaceCount][i] * weightedScore
      i += 1
    return scoreAdder

def yahtzee_weighted_score(diceIn):
  sameFaces = 0
  numberOfEachArray = convert_diceArray_to_faceCountArray(diceIn)
  i = 0
  while i < 5:
    if numberOfEachArray[i] > sameFaces:
      sameFaces = numberOfEachArray[i]
    i += 1
  weightedScore = 0
  if sameFaces == 5:
    weightedScore = 50
  else:
    if rollsRemaining == 2:
      weightedScore = 50 * upperSectionAndKindsTwoRolls[sameFaces][5]
    if rollsRemaining == 1:
      weightedScore = 50 * upperSectionAndKindsNextRoll[sameFaces][5]
  return weightedScore

#keep the most of the dice faces, return array with faces of the most and 0 to reroll
def yahtzee_keep_desired_dice(diceIn):
  numberOfEachArray = convert_diceArray_to_faceCountArray(diceIn)
  # try from ones -> sixes to track which face has the most, if the same number, keep the higher number
  most = -1 #track  most of each number
  whichFace = 0 #track which face has most
  i = 0
  while i < 6: # go through number of each dice array
    if numberOfEachArray[i] >= most: #see if current count is highest
      most = numberOfEachArray[i] #assign that count to most (most = which dice face has the most)
      whichFace = i + 1
    i += 1
  i = 0
  diceOut = [0,0,0,0,0]
  while i < most:
    diceOut[i] = whichFace
    i += 1
  return diceOut

###### testing #############
# dice = initial_array()
# print (dice)
# rollsRemaining = 2
# print (yahtzee_weighted_score(dice))
# rollsRemaining = 1
# print (yahtzee_weighted_score(dice))
# print (yahtzee_keep_desired_dice(dice))


######## testing #####################
# dice = initial_array()
# print (dice)
# rollsRemaining = 2
# print (kinds_highest_number_of_faces_and_face_total(dice))

# print ("desired dice", kinds_keep_desired_dice(dice))

# print (three_kind_score_weight(dice))


# Straights

In [5]:
def small_straight_state(diceIn):
  haveSstArray = [[1,1,1,1,0,0],[0,1,1,1,1,0],[0,0,1,1,1,1]]
  thisArray = convert_diceArray_to_faceCountArray(diceIn)#convert to face count
  thisArray = convert_faceCountArray_to_haveDontHaveArray(thisArray)
  matchCount = 0
  outArray = [0,0,0,0,0,0]
  state = 0 
  maxArray = [0,0,0,0,0,0]
  keepArray = [0,0,0,0,0,0]
  weightArray = [1,2,3,3,2,1]
  i = 0
  j = 0
  while i < 3:
    outArray = [0,0,0,0,0,0]
    while j < 6:
      if thisArray[j] == 1 and haveSstArray[i][j] == 1:
          outArray[j] = 1
          matchCount += weightArray[j]
      j += 1
    if matchCount > state:
      state = matchCount
      maxArray = outArray
    j = 0
    matchCount = 0
    i += 1 
  return state

def small_straight_weighted_score(diceIn):
  dice = diceIn
  diceState = small_straight_state(dice)
  straightProbability = 0
  if rollsRemaining == 2:
    straightProbability = smallStraightTwoRolls[diceState]
  if rollsRemaining == 1:
    straightProbability = smallStraightNextRoll[diceState]
  weightedScore = straightProbability * 30
  return weightedScore

def keep_small_straight(diceIn): #keep desired dice
  haveSstArray = [[1,1,1,1,0,0],[0,1,1,1,1,0],[0,0,1,1,1,1]]
  startingDice = diceIn
  thisArray = convert_diceArray_to_faceCountArray(startingDice)#convert to face count
  thisArray = convert_faceCountArray_to_haveDontHaveArray(thisArray)
  matchCount = 0
  outArray = [0,0,0,0,0,0]
  maxCount = 0 
  maxArray = [0,0,0,0,0,0]
  keepArray = [0,0,0,0,0,0]
  weightArray = [1,2,3,3,2,1]
  i = 0
  j = 0
  while i < 3:
    outArray = [0,0,0,0,0,0]
    while j < 6:
      if thisArray[j] == 1 and haveSstArray[i][j] == 1:
          outArray[j] = 1
          matchCount += weightArray[j]
      j += 1
    if matchCount > maxCount:
      maxCount = matchCount
      maxArray = outArray
    outArray = [0,0,0,0,0,0]
    j = 0
    matchCount = 0
    i += 1  
  keptDice = convert_faceCountArray_to_diceArray(maxArray)
  return keptDice

def large_straight_state(diceIn):
  haveLstArray = [[1,1,1,1,1,0],[0,1,1,1,1,1]]
  thisArray = convert_diceArray_to_faceCountArray(diceIn)#convert to face count
  thisArray = convert_faceCountArray_to_haveDontHaveArray(thisArray)
  matchCount = 0
  outArray = [0,0,0,0,0,0]
  state = 0 
  maxArray = [0,0,0,0,0,0]
  keepArray = [0,0,0,0,0,0]
  weightArray = [1,2,2,2,2,1]
  i = 0
  j = 0
  while i < 2:
    outArray = [0,0,0,0,0,0]
    while j < 6:
      if thisArray[j] == 1 and haveLstArray[i][j] == 1:
          outArray[j] = 1
          matchCount += weightArray[j]
      j += 1
    # print ("Out array ", i, " = ", outArray, " matchCount = ", matchCount)
    if matchCount > state:
      state = matchCount
      maxArray = outArray
    j = 0
    matchCount = 0
    i += 1 
  keptDice = convert_faceCountArray_to_diceArray(maxArray)
  return state

def large_straight_weighted_score(diceIn):
  dice = diceIn
  diceState = large_straight_state(dice)
  straightProbability = 0
  if rollsRemaining == 2:
    straightProbability = largeStraightTwoRolls[diceState]
  if rollsRemaining == 1:
    straightProbability = largeStraightNextRoll[diceState]
  weightedScore = straightProbability *40
  return weightedScore

def keep_large_straight(diceIn): #keep desired dice
  haveLstArray = [[1,1,1,1,1,0],[0,1,1,1,1,1]]
  thisArray = convert_diceArray_to_faceCountArray(diceIn)#convert to face count
  thisArray = convert_faceCountArray_to_haveDontHaveArray(thisArray)
  matchCount = 0
  outArray = [0,0,0,0,0,0]
  maxCount = 0 
  maxArray = [0,0,0,0,0,0]
  keepArray = [0,0,0,0,0,0]
  weightArray = [1,2,2,2,2,1]
  i = 0
  j = 0
  while i < 2:
    outArray = [0,0,0,0,0,0]
    while j < 6:
      if thisArray[j] == 1 and haveLstArray[i][j] == 1:
          outArray[j] = 1
          matchCount += weightArray[j]
      j += 1
    if matchCount > maxCount:
      maxCount = matchCount
      maxArray = outArray
    outArray = [0,0,0,0,0,0]
    j = 0
    matchCount = 0
    i += 1  
  keptDice = convert_faceCountArray_to_diceArray(maxArray)
  return keptDice

############ testing ###############################
# dice = initial_array()
# print (dice)
# rollsRemaining = 2
# print ("small st. weighted score :",small_straight_weighted_score(dice))
# print ("dice to keep :",keep_small_straight(dice))
# print ("large st. weighted score :",large_straight_weighted_score(dice))
# print ("dice to keep :",keep_large_straight(dice))
# rollsRemaining = 1
# print ("small st. weighted score :",small_straight_weighted_score(dice))
# print ("dice to keep :",keep_small_straight(dice))
# print ("large st. weighted score :",large_straight_weighted_score(dice))
# print ("dice to keep :",keep_large_straight(dice))


# Chance

In [6]:
def chance_weighted_score(diceIn):
  score = 0
  i = 0
  if rollsRemaining == 2:
    while i < 5:
      if diceIn[i] < 5: #keep dice 5 or higher
        if rollsRemaining == 2:
          score += 4.25 # average score if rerolling a dice two times
      if diceIn[i] >= 5:
        score += diceIn[i]
      i += 1
  if rollsRemaining == 1:
    while i < 5:
      if diceIn[i] < 4: #keep dice 4 or higher
        score += 3.5 # average score if rerolling one dice
      if diceIn[i] >= 4:
        score += diceIn[i]
      i += 1
  return score

def chance_keep_dice(diceIn):
  keepingDice = [0,0,0,0,0]
  if rollsRemaining == 2:
    i = 0
    while i < 5:
      if diceIn[i] >= 5: #keep dice 5 or higher on first roll
        keepingDice[i] = diceIn[i]
      else:
        keepingDice[i] = 0
      i += 1
  if rollsRemaining == 1:
    i = 0
    while i < 5:
      if diceIn[i] >= 4: #keep dice 4 or higher on second roll
        keepingDice[i] = diceIn[i]
      else:
        keepingDice[i] = 0
      i += 1
  return keepingDice


########## testing ################
# dice = initial_array()

# print (dice)
# rollsRemaining = 1
# print (chance_weighted_score(dice))
# print (chance_keep_dice(dice))
# rollsRemaining = 2
# print (chance_weighted_score(dice))
# print (chance_keep_dice(dice))




# Full House


In [7]:
def full_house_state(diceIn):
  #track current hand scoring
  ones = 0
  twos = 0
  threes = 0
  fours = 0
  fives = 0
  sixes = 0
  fullHouseState = [0,0,0,0,0]
  #loop through array of dice, track how many of each number there are
  i = 0
  while i < 5:
    if diceIn[i] == 1:
      ones +=1
    if diceIn[i] == 2:
      twos +=1
    if diceIn[i] == 3:
      threes +=1
    if diceIn[i] == 4:
      fours +=1
    if diceIn[i] == 5:
      fives +=1
    if diceIn[i] == 6:
      sixes +=1
    i += 1
  i = 0
  fullHouse = 0
  oneTwo = 0
  twoTwo = 0
  oneOrMoreThree = 0
  nothing = 0
  haveOneTwo = False #track if have one pair, so notify to see if have two pairs
  scoresArray = [ones,twos,threes,fours,fives,sixes]
  j = 0
  while j < 6:
    if scoresArray[j] == 2:
      oneTwo = 1
      if haveOneTwo == True:
        twoTwo = 1
        oneTwo = 0
      scoresArray[j] = 0
      haveOneTwo = True
    j += 1
  if ones >= 3 or twos >= 3 or threes >= 3 or fours >= 3 or fives >= 3 or sixes >= 3:
    oneOrMoreThree = 1
  if ones == 3 or twos == 3 or threes == 3 or fours == 3 or fives == 3 or sixes == 3:
    if ones == 2 or twos == 2 or threes == 2 or fours == 2 or fives == 2 or sixes == 2:
      fullHouse = 1
      oneOrMoreThree = 0
      oneTwo = 0
  if oneTwo == 0 and twoTwo == 0 and oneOrMoreThree == 0 and fullHouse == 0:
    nothing = 1
  fullHouseState = [nothing,oneTwo,oneOrMoreThree,twoTwo,fullHouse]
  return fullHouseState

def full_house_state_to_weighted_score(diceState):
  state = 0
  i = 0
  weight = 0
  probability = 0
  while i < 5:
    if diceState[i] == 1:
      state = i
    i += 1
  if rollsRemaining == 1:
    probability = fullHouseNextRoll[state]
  if rollsRemaining == 2:
    probability = fullHouseTwoRolls[state]
  weight = probability * 25
  return weight

def full_house_weighted_score(diceIn):
  fullHouseState = full_house_state(diceIn)
  return full_house_state_to_weighted_score(fullHouseState)

def full_house_keep_dice(diceIn):
    faceCountArray = convert_diceArray_to_faceCountArray(diceIn)
    j = 0
    while j < 6: # loop through each element in face Count array
      if faceCountArray[j] < 2:# if less than two, keep none
        faceCountArray[j] = 0
      if faceCountArray[j] == 2:
        faceCountArray[j] = 2 #if two of face, keep two
      if faceCountArray[j] >= 3:
        faceCountArray[j] = 3 #if three or more, keep three
      j += 1
    j = 0
    keepDiceArray = convert_faceCountArray_to_diceArray(faceCountArray)
    return keepDiceArray

########## testing #####################################
# dice = initial_array()
# rollsRemaining = 2
# print (dice)
# print (full_house_state(dice))
# print (full_house_weighted_score(dice))
# print (full_house_keep_dice(dice))
# rollsRemaining = 1
# print (full_house_weighted_score(dice))



# Scoring

In [8]:
#input dice and return array with scoring eg. ones score, full house score (0 or 25)...
def scoring(diceScoresIn):
#track current hand scoring
  ones = 0
  twos = 0
  threes = 0
  fours = 0
  fives = 0
  sixes = 0
  sST = 0
  lST = 0
  fH = 0
  chan = 0
  threekind = 0
  fourkind = 0
  yahtzee = 0
  #loop through array of dice, track how many of each number there are
  i = 0
  while i < 5:
    if diceScoresIn[i] == 1:
      ones +=1
    if diceScoresIn[i] == 2:
      twos +=1
    if diceScoresIn[i] == 3:
      threes +=1
    if diceScoresIn[i] == 4:
      fours +=1
    if diceScoresIn[i] == 5:
      fives +=1
    if diceScoresIn[i] == 6:
      sixes +=1
    i += 1
#################################################
###below calculates if score is reached or not###
#################################################
  #chance scoring. 
  chan = ones + (twos * 2) + (threes * 3) + (fours * 4) + (fives * 5) + (sixes * 6)
  #three of a kind scoring
  if ones >= 3 or twos >= 3 or threes >= 3 or fours >= 3 or fives >= 3 or sixes >= 3:
    threekind = chan
  #four of a kind scoring
  if ones >= 4 or twos >= 4 or threes >= 4 or fours >= 4 or fives >= 4 or sixes >= 4:
    fourkind = chan
  #yahtzee scoring
  if ones >= 5 or twos >= 5 or threes >= 5 or fours >= 5 or fives >= 5 or sixes >= 5:
    yahtzee = 50
  #fullhouse scoring
  if ones == 3 or twos == 3 or threes == 3 or fours == 3 or fives == 3 or sixes == 3:
    if ones == 2 or twos == 2 or threes == 2 or fours == 2 or fives == 2 or sixes == 2:
      fH = 25
  #large straight scoring
  if ones == 1 and twos == 1 and threes == 1 and fours == 1 and fives == 1:
    lST = 40
  if twos == 1 and threes == 1 and fours == 1 and fives == 1 and sixes == 1:
    lST = 40
  #small straight scoring
  if ones >= 1 and twos >= 1 and threes >= 1 and fours >= 1:
    sST = 30
  if twos >= 1 and threes >= 1 and fours >= 1 and fives >= 1:
    sST = 30
  if threes >= 1 and fours >= 1 and fives >= 1 and sixes >= 1:
    sST = 30
  CountOut = [ones*1, twos*2, threes*3, fours*4, fives*5, sixes*6, threekind, fourkind, fH, sST, lST, chan, yahtzee]
  return(CountOut)

def keep_largest_score(scoresIn):
  i = 0
  highScore = -1
  scoreUsing = 0
  while i < 13:
    if scoresIn[i] > highScore:
      highScore = scoresIn[i]
      scoreUsing = i
    i += 1
  return scoreUsing


####### testing ############
# dice = initial_array()
# print (dice)
# print (scoring(dice))
# print (keep_largest_score(scoring(dice)))
# print (keep_largest_score(final_score_selected_weights(scoring(dice))))



# Weights


In [9]:
thisRoundHandWeights = [4,2.5,1.8,1.4,1.2,1,1.5,2,.8,.9,1.5,.1,5] #weights to use for this hand testing
scoresUsed = [1,1,1,1,1,1,1,1,1,1,1,1,1]
#take in dice, return dice array to keep based on weighted weights
def all_in_one_weights(dice):
  #first, get weights for all possible scores
  theseDice = dice
  weightsScoringArray = [0,0,0,0,0,0,0,0,0,0,0,0,0]#scoring weights or each possible score
  keepDesiredDiceArray = [0,0,0,0,0,0,0,0,0,0,0,0,0]#dice to keep for each possible scoring attempt
  upperSectionWeights = faces_weighted_scores(theseDice)
  upperSectionKeepDice = keep_desired_face(dice)
  i = 0
  while i < 6:
    weightsScoringArray[i] = upperSectionWeights[i]
    keepDesiredDiceArray[i] = upperSectionKeepDice[i]
    i += 1
  weightsScoringArray[6] = three_kind_score_weight(theseDice)
  keepDesiredDiceArray[6] = kinds_keep_desired_dice(theseDice)#three kind
  weightsScoringArray[7] = four_kind_score_weight(theseDice)
  keepDesiredDiceArray[7] = kinds_keep_desired_dice(theseDice)#four kind
  weightsScoringArray[8] = full_house_weighted_score(theseDice)
  keepDesiredDiceArray[8] = full_house_keep_dice(theseDice)
  weightsScoringArray[9] = small_straight_weighted_score(theseDice)
  keepDesiredDiceArray[9] = keep_small_straight(theseDice)
  weightsScoringArray[10] = large_straight_weighted_score(theseDice)
  keepDesiredDiceArray[10] = keep_large_straight(theseDice)
  weightsScoringArray[11] = chance_weighted_score(theseDice)
  keepDesiredDiceArray[11] = chance_keep_dice(theseDice)
  weightsScoringArray[12] = yahtzee_weighted_score(theseDice)
  keepDesiredDiceArray[12] = yahtzee_keep_desired_dice(theseDice)
  # print("Weight scoring array : ", weightsScoringArray)
  #next, take all weights, adjust for handWeights, zero out used scores
  adjustedWeights = [0,0,0,0,0,0,0,0,0,0,0,0,0]
  i = 0
  while i < 13:
    adjustedWeights[i] = weightsScoringArray[i] * thisRoundHandWeights[i] *scoresUsed[i]
    i += 1
  #find highest weight, return dice array with highest weight
  # print("Adjusted weights : ", adjustedWeights)
  # print("scores used after adjusted weights : ", scoresUsed)
  # print("weight scoring array after adjusted weights : ", weightsScoringArray)
  highestWeight = -1
  weightValue = 0 #track which hand has the highest weight
  i = 0
  while i < 13:
    if adjustedWeights[i] > highestWeight:
      highestWeight = adjustedWeights[i]
      weightValue = i
    i += 1
  keepDiceArray = keepDesiredDiceArray[weightValue] #return diceArray of dice to keep
  return keepDiceArray

#play a single hand using all in one weights
#must adjust and reset number of rerolls (to properly get weights)
def playSingleHand():
  rollsRemaining = 2
  dice = initial_array()
  # print("initial dice", dice)
  keepDice = all_in_one_weights(dice)
  # print("keep first dice : ", keepDice)
  rerolledDice = get_new_dice(keepDice)
  # print("dice after first reroll : ", rerolledDice)
  rollsRemaining = 1
  secondKeepDice = all_in_one_weights(rerolledDice)
  # print("keep second dice : ", secondKeepDice)
  finalRerolledDice = get_new_dice(secondKeepDice)
  # print("final dice : ", finalRerolledDice)
  rollsRemaining = 2
  return finalRerolledDice




adjustedScoreSelectWeights = [3,2.2,1.8,1.4,1.1,1,.7,.8,.8,1,1,.1,1]
#take in dice, get scores, weigh scores, adjust weights for selecting final score to use, select highest weighted score, negate used score for next hand
def final_score_selected_weights(diceScoresIn):
#input dice and return array with scoring eg. ones score, full house score (0 or 25)...
#track current hand scoring
  ones = 0
  twos = 0
  threes = 0
  fours = 0
  fives = 0
  sixes = 0
  sST = 0
  lST = 0
  fH = 0
  chan = 0
  threekind = 0
  fourkind = 0
  yahtzee = 0
  #loop through array of dice, track how many of each number there are
  i = 0
  while i < 5:
    if diceScoresIn[i] == 1:
      ones +=1
    if diceScoresIn[i] == 2:
      twos +=1
    if diceScoresIn[i] == 3:
      threes +=1
    if diceScoresIn[i] == 4:
      fours +=1
    if diceScoresIn[i] == 5:
      fives +=1
    if diceScoresIn[i] == 6:
      sixes +=1
    i += 1
#################################################
###below calculates if score is reached or not###
#################################################
  #chance scoring. 
  chan = ones + (twos * 2) + (threes * 3) + (fours * 4) + (fives * 5) + (sixes * 6)
  #three of a kind scoring
  if ones >= 3 or twos >= 3 or threes >= 3 or fours >= 3 or fives >= 3 or sixes >= 3:
    threekind = chan
  #four of a kind scoring
  if ones >= 4 or twos >= 4 or threes >= 4 or fours >= 4 or fives >= 4 or sixes >= 4:
    fourkind = chan
  #yahtzee scoring
  if ones >= 5 or twos >= 5 or threes >= 5 or fours >= 5 or fives >= 5 or sixes >= 5:
    yahtzee = 50
  #fullhouse scoring
  if ones == 3 or twos == 3 or threes == 3 or fours == 3 or fives == 3 or sixes == 3:
    if ones == 2 or twos == 2 or threes == 2 or fours == 2 or fives == 2 or sixes == 2:
      fH = 25
  #large straight scoring
  if ones == 1 and twos == 1 and threes == 1 and fours == 1 and fives == 1:
    lST = 40
  if twos == 1 and threes == 1 and fours == 1 and fives == 1 and sixes == 1:
    lST = 40
  #small straight scoring
  if ones >= 1 and twos >= 1 and threes >= 1 and fours >= 1:
    sST = 30
  if twos >= 1 and threes >= 1 and fours >= 1 and fives >= 1:
    sST = 30
  if threes >= 1 and fours >= 1 and fives >= 1 and sixes >= 1:
    sST = 30
  scoresOut = [ones*1, twos*2, threes*3, fours*4, fives*5, sixes*6, threekind, fourkind, fH, sST, lST, chan, yahtzee]
 

 
  #weighted scores: 0-5 = faces 1-6, 6 = 3x, 7 = 4x, 8 = Full House, 9 = Sm Straight, 10 = Lg Straight, 11 = Chance, 12 = Yahtzee
  scoreSelectOutputWeights = [0,0,0,0,0,0,0,0,0,0,0,0,0]
  i = 0
  while i < 13:
    scoreSelectOutputWeights[i] = scoresOut[i]*adjustedScoreSelectWeights[i]*scoresUsed[i]
    i += 1
  #set used scores to negative value so they can't possibly be used as the highest value
  #if several scores are 0
  i = 0
  while i < 13:
    if scoresUsed[i] == -1:
      scoreSelectOutputWeights[i] = adjustedScoreSelectWeights[i]*scoresUsed[i]
    i+=1
  # print("Weighted score selection : ", scoreSelectOutputWeights)
  #select highest weight for final score to use
  i = 0
  highScore = -1
  scoreUsing = 0 #which score to use eg. 0 = 1s -> 12 = Yahtzee
  while i < 13:
    if scoreSelectOutputWeights[i] > highScore:
      highScore = scoreSelectOutputWeights[i]
      scoreUsing = i
    i += 1



















  ### if multiple scores are 0 and need to select which score to use
  # calculate how many possible to select zero scores there are
  totalUnusedScoresRemaining = 0
  if highScore == 0:
    #select which zero score to use first
    #0-5 = faces 1-6, 6 = 3x, 7 = 4x, 8 = Full House, 9 = Sm Straight, 10 = Lg Straight, 11 = Chance, 12 = Yahtzee
    zeroScoreSelectFirst = [0,1,2,12,7,10,3,4,9,8,5,6,11]
    zeroButtons = [] #track which buttons still have zero score
    itr = 0
    while itr < 13:
      if scoreSelectOutputWeights[itr] == 0 or scoreSelectOutputWeights[itr] == 0.0:
        zeroButtons.append(itr)
      itr += 1
    buttonsLeft = len(zeroButtons)
    thisbuttonTracking = 0
    highestNumberPassed = 100 #track how many have been tried and take lowest
    thisNumberPassed = 0
    if buttonsLeft >= 2:
      for j in range(buttonsLeft):
        thisbuttonTracking = zeroButtons[j]
        for i in range(13):
          if zeroScoreSelectFirst[i] == thisbuttonTracking:
            thisNumberPassed = i
            if thisNumberPassed < highestNumberPassed:
              highestNumberPassed = thisNumberPassed
      scoresUsed[thisbuttonTracking] = -1
      return thisbuttonTracking, scoresOut[thisbuttonTracking]
  
  
  scoresUsed[scoreUsing] = -1
  # print("scores used : ", scoresUsed)
  # print("scores : ", scoresOut)
  return scoreUsing, scoresOut[scoreUsing]
  

















# finalDice = playSingleHand()
# scores = final_score_selected_weights(finalDice)
# print("final scores : ", scores)
arrayOfScores = [] #save array of scores to analize
upperBonuses = 0
def play_full_round():
  roundScore = [0,0,0,0,0,0,0,0,0,0,0,0,0]
  i = 0
  while i < 13:
    # print("Round : ", i)
    finalDice = playSingleHand()
    scores = final_score_selected_weights(finalDice)
    scoreUsed = scores[0]
    thatScore = scores[1]
    roundScore[scoreUsed] = thatScore
    i += 1
  #   print("Used hand scores = ", roundScore)
  # print("Final Round scores = ", roundScore)
  upperBonusAchieved = 0
  i = 0
  while i<6:
    upperBonusAchieved += roundScore[i]
    i += 1
  totalScore = 0
  i = 0
  while i < 13:
    totalScore += roundScore[i]
    i += 1
  if upperBonusAchieved >= 63:
    totalScore += 35
    global upperBonuses
    upperBonuses += 1
  #   print ("upper bonus achieved")
  arrayOfScores.append(roundScore)
  return totalScore
  

def play_round_of_rounds(rounds):
  i = 0
  scoreSum = 0
  while i < rounds:
    scoreSum += play_full_round()
    global scoresUsed #reset global scores used
    scoresUsed = [1,1,1,1,1,1,1,1,1,1,1,1,1] #reset used scores
    i+=1
  averageScore = scoreSum/rounds
  return averageScore


adjustedScoreSelectWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
thisRoundHandWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
scoreSum = 0
rounds = 0
while rounds < 1000:
  scoreSum += play_round_of_rounds(1)
  rounds += 1
avgScore = scoreSum/rounds
# print (scoreSum)
print("avg score = ", avgScore)

i = 0
handAverages = [0,0,0,0,0,0,0,0,0,0,0,0,0]
while i < rounds:
  j = 0
  while j<13:
    handAverages[j] += arrayOfScores[i][j]
    j +=1
  i += 1
i = 0
while i < 13:
  handAverages[i] = handAverages[i]/rounds
  i += 1
print("Weights used : ", thisRoundHandWeights, adjustedScoreSelectWeights)
print ("Rounds played : ", rounds)
print("Upper bonuses : ", upperBonuses/rounds)
print("Yahtzee percent : ", handAverages[12]/50)
print ("Scoring averages : ", handAverages)




avg score =  212.495
Weights used :  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5] [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Rounds played :  1000
Upper bonuses :  0.026
Yahtzee percent :  0.297
Scoring averages :  [1.928, 4.092, 6.42, 8.58, 10.81, 13.284, 22.922, 16.826, 23.65, 29.94, 36.04, 22.243, 14.85]


# Game play

In [10]:
#adjust weights to test. Take in base array, output 10 adjusted arrays
def create_adjusted_weights_array(weightAdjustment,baseArray):
  outArrays = []#arrays to return with adjusted weights
  i = 0
  while i < 300:
    j = 0
    adjustedArray = [0,0,0,0,0,0,0,0,0,0,0,0,0] #this adjusted array
    while j < 13:
      #select a random weight. +,-,0
      weightChoice = [0,(-1*round(random.uniform(0.03, .25),2)),round(random.uniform(0.03, .25),2)]
      upDownZero = random.choice(weightChoice)
      newWeight = baseArray[j] + upDownZero
      if newWeight <= 0.01:
        newWeight = 0.01
      if newWeight >= 1:
        newWeight = 1
      adjustedArray[j] = round(newWeight,4)
      j+=1
    outArrays.append(adjustedArray)
    i+=1
  return outArrays

def create_wild_adjusted_weights_array(arraysToMake):
  outArrays = []#arrays to return with adjusted weights
  i = 0
  while i < arraysToMake:
    j = 0
    adjustedArray = [0,0,0,0,0,0,0,0,0,0,0,0,0] #this adjusted array
    while j < 13:
      adjustedArray[j] = round(random.uniform(0.10, .90),2)
      j+=1
    outArrays.append(adjustedArray)
    i+=1
  return outArrays

def create_each_score_adjusted_weights_array():
  outArrays = []#arrays to return with adjusted weights
  i = 0
  while i < 130:
    j = 0
    adjustedArray = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
    while j < 13:
      k = 0
      while k < 13:
        if j == k:
          m = .1
          while m < 1:
            thisArray[k] = m
            m += .1
      j+=1
    outArrays.append(adjustedArray)
    i+=1
  return outArrays

def adjust_weights():
  global adjustedScoreSelectWeights 
  global thisRoundHandWeights
  currentWinninghandWeight = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5] #change to current best weight
  currentWinningScoreWeight = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5] #change to current best weight
  handScoringTracker = []
  scoreSelectScoringTracker = []
  rounds = 0
  weightsChecking = 100 #adjust number of weights per child
  roundsToTestPerWeight = 5000
  winningHandWeightScore = 0
  winningScoreWeightScore = 0
  amountToAdjustWeights = round(random.uniform(0.03, .15),2)
  # amountToAdjustWeights = 0.02
  while rounds < 10:
    print("--------- Round : ", rounds+1,"---------------------")
    #adjust hand select weights
    weightArrays = create_adjusted_weights_array(round(random.uniform(0.03, .15),2),currentWinninghandWeight)#create weight arrays to test
    arrayScores = []
    i = 0
    while i < weightsChecking:
      thisRoundHandWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      thisRoundHandWeights = weightArrays[i]
      thisWeightScore = 0
      arrayScores.append(play_round_of_rounds(roundsToTestPerWeight))#send how many rounds to play
      # print ("Score : ",arrayScores[i], " hand weights used : ", weightArrays[i] )
      i+=1
    #save best weight array
    bestScore = 0
    bestArray = 0
    i = 0
    while i < weightsChecking:
      if arrayScores[i] > bestScore:
        bestScore = arrayScores[i]
        bestArray = i
      i += 1
    print("(hand select adjust)Highest score : ", arrayScores[bestArray], " array : ",weightArrays[bestArray])
    #assign winning array if better than current best array
    if arrayScores[bestArray] > winningHandWeightScore:
      winningHandWeightScore = arrayScores[bestArray]
      currentWinninghandWeight = weightArrays[bestArray]
      print("New hand select weight assigned - Score ",winningHandWeightScore)
    handScoringTracker.append(bestScore)

    #adjust scoring select weights
    weightScoreArrays = create_adjusted_weights_array(amountToAdjustWeights,currentWinningScoreWeight)#create weight arrays to test
    arrayScores = []
    i = 0
    while i < weightsChecking:
      adjustedScoreSelectWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      adjustedScoreSelectWeights = weightArrays[i]
      thisWeightScore = 0
      arrayScores.append(play_round_of_rounds(roundsToTestPerWeight))#send how many rounds to play
      # print ("(score select adjust) Score  : ",arrayScores[i], " hand weights used : ", weightArrays[i] )
      i+=1
    #save best weight array
    bestScore = 0
    bestArray = 0
    i = 0
    while i < weightsChecking:
      if arrayScores[i] > bestScore:
        bestScore = arrayScores[i]
        bestArray = i
      i += 1
    print ("(score select) Winning score : ",arrayScores[bestArray]," winning weight :",weightArrays[bestArray] )
    #assign winning array if better than current best array
    if arrayScores[bestArray] > winningScoreWeightScore:
      winningScoreWeightScore = arrayScores[bestArray]
      currentWinningScoreWeight = weightArrays[bestArray]
      print("New score select weight assigned - score ",winningScoreWeightScore )
    scoreSelectScoringTracker.append(bestScore)
    rounds += 1
  print("winning hand select weight: ", currentWinninghandWeight )
  print("winning score select weight: ", currentWinningScoreWeight)
  print("hand score tracker : ",handScoringTracker)
  print ("scoreSelect score tracker : ", scoreSelectScoringTracker)

def adjust_both_weights():
  global adjustedScoreSelectWeights 
  global thisRoundHandWeights
  currentWinninghandWeight = [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43] #change to current best weight
  currentWinningScoreWeight = [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43] #change to current best weight
  handScoringTracker = []
  scoreSelectScoringTracker = []
  rounds = 0
  weightsChecking = 300 #adjust number of weights per child
  roundsToTestPerWeight = 10000
  winningHandWeightScore = 0
  winningScoreWeightScore = 0
  amountToAdjustWeights = round(random.uniform(0.03, .15),2)
  # amountToAdjustWeights = 0.02
  while rounds < 10:
    print("--------- Round : ", rounds+1,"---------------------")
    #adjust hand select weights
    weightArrays = create_adjusted_weights_array(round(random.uniform(0.03, .15),2),currentWinninghandWeight)#create weight arrays to test
    arrayScores = []
    i = 0
    while i < weightsChecking:
      thisRoundHandWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      thisRoundHandWeights = weightArrays[i]
      adjustedScoreSelectWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      adjustedScoreSelectWeights = weightArrays[i]
      thisWeightScore = 0
      arrayScores.append(play_round_of_rounds(roundsToTestPerWeight))#send how many rounds to play
      # print ("Score : ",arrayScores[i], " hand weights used : ", weightArrays[i] )
      i+=1
    #save best weight array
    bestScore = 0
    bestArray = 0
    i = 0
    while i < weightsChecking:
      if arrayScores[i] > bestScore:
        bestScore = arrayScores[i]
        bestArray = i
      i += 1
    print("(hand select adjust)Highest score : ", arrayScores[bestArray], " array : ",weightArrays[bestArray])
    #assign winning array if better than current best array
    if arrayScores[bestArray] > winningHandWeightScore:
      winningHandWeightScore = arrayScores[bestArray]
      currentWinninghandWeight = weightArrays[bestArray]
      print("New hand select weight assigned - Score ",winningHandWeightScore)
    handScoringTracker.append(bestScore)

    print ("(score select) Winning score : ",arrayScores[bestArray]," winning weight :",weightArrays[bestArray] )
    #assign winning array if better than current best array
    if arrayScores[bestArray] > winningScoreWeightScore:
      winningScoreWeightScore = arrayScores[bestArray]
      currentWinningScoreWeight = weightArrays[bestArray]
      print("New score select weight assigned - score ",winningScoreWeightScore )
    scoreSelectScoringTracker.append(bestScore)
    rounds += 1
  print("winning hand select weight: ", currentWinninghandWeight )
  print("winning score select weight: ", currentWinningScoreWeight)
  print("hand score tracker : ",handScoringTracker)
  print ("scoreSelect score tracker : ", scoreSelectScoringTracker)

def wild_adjust_both_weights():
  global adjustedScoreSelectWeights 
  global thisRoundHandWeights
  currentWinninghandWeight = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5] #change to current best weight
  currentWinningScoreWeight = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5] #change to current best weight
  handScoringTracker = []
  scoreSelectScoringTracker = []
  rounds = 0
  winningHandWeightScore = 0
  amountToAdjustWeights = 0.02
  arraysToMake = 1000
  allScoresandWeights = []
  while rounds < 1:
    #adjust hand select weights
    weightArrays = create_wild_adjusted_weights_array(arraysToMake)#create weight arrays for hands to test
    weightScoreArrays = create_wild_adjusted_weights_array(arraysToMake)#create weight arrays for scoring to test
    arrayScores = []
    i = 0
    while i < arraysToMake:
      thisRoundHandWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      adjustedScoreSelectWeights = [.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5,.5]
      thisRoundHandWeights = weightArrays[i]
      adjustedScoreSelectWeights = weightScoreArrays[i]
      thisWeightScore = 0
      avgScore = play_round_of_rounds(5000)
      arrayScores.append(avgScore)#send how many rounds to play
      # print ("Score : ",arrayScores[i], " hand weights used : ", weightArrays[i], " score weights used : ",weightScoreArrays[i] )
      i+=1
      info = avgScore,thisRoundHandWeights,adjustedScoreSelectWeights
      allScoresandWeights.append(info)
      print(info)
    #save best weight array
    bestScore = 0
    bestArray = 0
    i = 0
    while i < arraysToMake:
      if arrayScores[i] > bestScore:
        bestScore = arrayScores[i]
        bestArray = i
      i += 1
    print("Highest score : ", arrayScores[bestArray], " hand array : ",weightArrays[bestArray], " score select array : ", weightScoreArrays[bestArray])
    #assign winning array if better than current best array
    if arrayScores[bestArray] > winningHandWeightScore:
      winningHandWeightScore = arrayScores[bestArray]
      currentWinninghandWeight = weightArrays[bestArray]
      currentWinningScoreWeight = weightScoreArrays[bestArray]
      print("New weight assigned")
    handScoringTracker.append(bestScore)
    scoreSelectScoringTracker.append(bestScore)
    print ("- - - - - - - - - - - - - - - - - - ")
    rounds += 1
    print("winning hand select weight: ", currentWinninghandWeight )
    print("winning score select weight: ", currentWinningScoreWeight)
    print("hand score tracker : ",handScoringTracker)
    print ("scoreSelect score tracker : ", scoreSelectScoringTracker)
    f = open('runs2.csv', 'w')

    with f:

        writer = csv.writer(f)
        writer.writerows(allScoresandWeights)

def test_arrays():
  global thisRoundHandWeights
  global adjustedScoreSelectWeights
  thisRoundHandWeights = [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43]
  adjustedScoreSelectWeights = [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43]
  roundsToPlay = 10000
  averageScore = 0
  averageScore = play_round_of_rounds(roundsToPlay)
  i = 0
  handAverages = [0,0,0,0,0,0,0,0,0,0,0,0,0]
  while i < roundsToPlay:
    j = 0
    while j<13:
      handAverages[j] += arrayOfScores[i][j]
      j +=1
    i += 1
  i = 0
  while i < 13:
    handAverages[i] = handAverages[i]/roundsToPlay
    i += 1
  print("Weights used : ", thisRoundHandWeights, adjustedScoreSelectWeights)
  print ("Rounds played : ", roundsToPlay)
  print ("Average score : ", averageScore)
  print ("Scoring averages : ", handAverages)
  

test_arrays()


Weights used :  [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43] [0.6, 1, 0.65, 0.6, 0.59, 0.46, 0.35, 0.64, 0.38, 0.28, 0.51, 0.28, 0.43]
Rounds played :  10000
Average score :  222.3503
Scoring averages :  [1.8892, 4.811, 6.9822, 10.0992, 12.8245, 15.168, 22.2646, 19.1292, 23.1025, 29.559, 34.28, 21.7463, 15.59]


In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
#   print('and then re-execute this cell.')
# else:
#   print(gpu_info)

# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
#   print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#   print('re-execute this cell.')
# else:
#   print('You are using a high-RAM runtime!')